In [3]:
# Goal:
## 1: pull 100 random public matches.
## 2: get match details from matches that meet analysis criteria.
## 3: get all player MMRs from selected matches (max 1000).
## 4: put those data into PostGres
# Relies on PostGres DB being up and running on localhost
import requests
import pandas as pd
import numpy as np
import psycopg2
import json
import time
import support_EventLogger as logger

In [4]:
process_name = 'ETL01_GetMatchResults.ipynb' # for logger
logger.log(process_name,'Process started, libraries loaded')

In [5]:
# get random set of 100 matches
logger.log(process_name,'Fetching public matches from opendota api')
response = requests.get("https://api.opendota.com/api/publicMatches")
public_match_results = pd.DataFrame.from_dict(response.json(), orient='columns')
logger.log(process_name,'Public match results fetched from opendota api')

In [6]:
# clean up Match data
## NaN MMR is unhelpful
## fewer than 10 players is unhelpful
public_match_results.dropna(inplace=True)

# Low-tech check for 5v5? Check for 4 commas (separates 5 hero values) on each team
public_match_results['commacount'] = public_match_results.apply(lambda x: x.radiant_team.count(',')+x.dire_team.count(','),axis=1)
public_match_resultsn = public_match_results[public_match_results['commacount'] == 8]
public_match_results.drop(['commacount'],axis=1, inplace=True)

# hero names not needed, these will be pulled in separately with Players later
public_match_results.drop(['radiant_team','dire_team'],axis=1,inplace=True)

In [7]:
batch_file_name = 'last_matchresults_batch.csv'
existing_match_ids_file_name = 'existing_match_ids.csv' # used to prevent the same match from going to postgres twice

In [8]:
# match data is prepared. time to move to Postgres
## get credentials from .config, connect to DB
logger.log(process_name,'Starting postgres connection')
credential_dict = {}
with open ('connections.config','r') as config_file:
    credential_dict = json.loads(config_file.read())

connection_string = "dbname="+credential_dict['psql_dbname'] + \
    " user="+credential_dict['psql_username'] + \
    " password="+credential_dict['psql_password']

conn = psycopg2.connect(connection_string)

# place the psql cursor
cur = conn.cursor()
logger.log(process_name,'Postgres connected')

In [9]:
# Need to check if match_ids of new matches are already in the database
with open (existing_match_ids_file_name,'w') as match_id_file:
    cur.copy_to(file=match_id_file,table='match_results',sep='|',columns=['match_id'])
# a copy of existing match_ids is kept as a csv just for reference on searches outside this project, not strictly needed
existing_match_ids = pd.read_csv(existing_match_ids_file_name,header=None,sep='|', names=['match_id'])
existing_match_ids['id_exists'] = True

#using a left join to connect any new matches with existing match_ids in postgres
public_match_results = public_match_results.merge(existing_match_ids,how='left',on='match_id')
public_match_results = public_match_results[public_match_results['id_exists'] != True] # only non-overlapping match results remain
public_match_results.drop(columns=['id_exists'],axis=1,inplace=True)

#write the results to a CSV, even if rows are empty
public_match_results.to_csv(batch_file_name,index=False,header=False,sep='|')

In [10]:
logger.log(process_name,'Batch prepared with shape ' + str(public_match_results.shape) + ', attempting to add to postgres')
if public_match_results.shape[0] == 0:
    logger.log(process_name,'Batch empty, no data to add to postgres')
else:
    # copy csv to postgres, commit the change
    with open (batch_file_name,'r') as batch_file:
        cur.copy_from(batch_file, 'match_results', sep='|')
    conn.commit()
    logger.log(process_name,'Data loaded, committed to postgres')

In [11]:
# close psql cursor and connection
cur.close()
conn.close()
logger.log(process_name,'Closed postgres connection')

In [12]:
logger.log(process_name,'Process ending')